In [1]:
import numpy as np

In [4]:
data_dir = "data/interim/compressed_tiles"
clean_dir = "data/clean"

In [3]:
cancer = 23.4394
capricorn = -23.4394

def ilat(lat):
    return round((10 - lat % 10) * 20) - 1

def trim_to_tropics(a):
    assert a.shape == (1200,7200)
    return a[ilat(cancer):(1000+ilat(capricorn)),:]

def combine_tiles(measure):
    b = None

    for lat in range(-20,40,10):
        a = np.load(f"{data_dir}/{lat}_-180_{measure}.npy")[0]
        for i in range(-170,180,10):
            a = np.hstack([a, np.load(f"{data_dir}/{lat}_{i}_{measure}.npy")[0]])
        
        b = a if b is None else np.vstack([a,b])
    
    return trim_to_tropics(b)

In [5]:
for measure in ["cover", "loss", "mask"]:
    # NB loss data currently only valid as binary
    # 0 is no loss
    # > 0 is loss (but actual actual figures are misleadingly compressed)
    b = combine_tiles(measure)
    np.save(f"{clean_dir}/{measure}_tropics.npy", b)

In [6]:
# NB loss data not yet compressed properly
loss_files = [combine_tiles(f"loss_{yr}") for yr in range(1,21)]

loss_stack = np.stack([np.zeros_like(loss_files[0]), *loss_files])
loss_max = np.max(loss_stack, axis=0)
loss_sum = np.sum(loss_stack, axis=0)
loss_year = np.argmax(loss_stack, axis=0)

np.save(f"{clean_dir}/loss_stack_tropics.npy", loss_stack)
np.save(f"{clean_dir}/loss_max_tropics.npy", loss_max)
np.save(f"{clean_dir}/loss_sum_tropics.npy", loss_sum)
np.save(f"{clean_dir}/loss_year_tropics.npy", loss_year)